In [3]:
# Import the Data
import json
from pprint import pprint

#Read in submissions
with open('r_data/RS_2014-12_speedrun.json', 'r', encoding='utf-8') as file:
    submission_dataset = json.loads(file.read())

# Read in comments
with open('r_data/RC_2014-12_speedrun.json', 'r', encoding='utf-8') as file:
    comment_dataset = json.loads(file.read())

file.close()
#pprint(type(submission_dataset))
print("First submission element \n")
pprint(submission_dataset[0])
print("\n")
print("First Comment element \n")
pprint(comment_dataset[0])

First submission element 

{'archived': True,
 'author': 'intheblue',
 'author_flair_css_class': '',
 'author_flair_text': 'Hotline Miami, Spider Solitare',
 'created': 1417395407,
 'created_utc': '1417395407',
 'distinguished': None,
 'domain': 'twitch.tv',
 'downs': 0,
 'edited': False,
 'from': None,
 'from_id': None,
 'from_kind': None,
 'gilded': 0,
 'hide_score': False,
 'id': '2nw69z',
 'is_self': False,
 'link_flair_css_class': None,
 'link_flair_text': None,
 'media': {'oembed': {'description': 'Recorded live on an hour ago',
                      'height': 300,
                      'html': '&lt;iframe class="embedly-embed" '
                              'src="//cdn.embedly.com/widgets/media.html?src=http%3A%2F%2Fwww.twitch.tv%2Fwidgets%2Flive_embed_player.swf%3Fchannel%3Ddingodroles&amp;fv=hostname%3Dwww.twitch.tv%26start_volume%3D25%26channel%3Ddingodrole%26auto_play%3Dfalse&amp;url=http%3A%2F%2Fwww.twitch.tv%2Fdingodrole%2Fc%2F5593916&amp;image=http%3A%2F%2Fstatic-cdn.jtv

In [4]:
# Data preporcessing part 1
# We curate the tags we want.       

# process submission
info = {}
submission_dataset_two = []
for dictionary in submission_dataset:
    info = {
        "id":            dictionary.get('id'),
        "title":         dictionary.get('title'),
        "selftext":      dictionary.get('selftext'),
        "score":         dictionary.get('score'),
        "num_comments":  dictionary.get('num_comments')
    }
    submission_dataset_two.append(info)
print("Submission 0:  ")
print(submission_dataset_two[0])
print("\n")

info_two = {}
comment_dataset_two = []
for dictionary in comment_dataset:
    info_two = {
        "parent_id": dictionary.get('parent_id'),
        "body":      dictionary.get('body'),
        "score":     dictionary.get('score')
    }
    comment_dataset_two.append(info_two)
print("Comment 0:  ")
print(comment_dataset_two[0])

with open('r_data/sub_data.json', 'w', encoding='utf-8') as file:
    json.dump(submission_dataset_two, file, ensure_ascii=False, indent=4)

with open('r_data/com_data.json', 'w', encoding='utf-8') as file:
    json.dump(comment_dataset_two, file, ensure_ascii=False, indent=4)
    
file.close()

Submission 0:  
{'id': '2nw69z', 'title': '[WR]Hotline Miami Any% NG in 20:16 by Dingodrole', 'selftext': '', 'score': 37, 'num_comments': 6}


Comment 0:  
{'parent_id': 't3_2nvw48', 'body': 'this only beats my PB time by around a minute :/ a TAS of this should be pretty close to 48:30 easily...', 'score': 6}


In [26]:
# Topic model LDA
# First attempt is to use Sub title + self text to generate topics
# Second attempt is to use just sub title + comments
# third attempt is just look at comments
# fourth is title + self text + comments
#Read in curated submissions
# Code by ... from: https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/ for Gensim model NOT USED
# Code by ... from: https://stackabuse.com/python-for-nlp-topic-modeling/ for LDA model

import pandas as pd
import numpy as np
import sys
import random

# shows the data
# submission_dataset_three.dropna()

######This is for Title only#####

# submission_dataset_three = pd.read_json(r'r_data/sub_data.json')

# from sklearn.feature_extraction.text import CountVectorizer

# count_vect = CountVectorizer(max_df=0.8, min_df=2, stop_words='english')
# doc_term_matrix = count_vect.fit_transform(submission_dataset_three['title'].values.astype('U'))

# from sklearn.decomposition import LatentDirichletAllocation

# LDA = LatentDirichletAllocation(n_components=5, random_state=42)
# LDA.fit(doc_term_matrix)


#####This is for title + selftext#####

with open('r_data/sub_data.json', 'r', encoding='utf-8') as file:
    submission_dataset_three = json.loads(file.read())

file.close()
corpus = {}
corpus_two = []

for dictionary in submission_dataset_three:
    corpus = {
        "data": dictionary.get('title') + dictionary.get('selftext')
        "id":            dictionary.get('id'),
        "score":         dictionary.get('score'),
        "num_comments":  dictionary.get('num_comments')
    }
    corpus_two.append(corpus)

with open('r_data/sub_title_selftext_data.json', 'w', encoding='utf-8') as file:
    json.dump(corpus_two, file, ensure_ascii=False, indent=4)

title_selftext = pd.read_json(r'r_data/sub_title_selftext_data.json')
    
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(max_df=0.8, min_df=2, stop_words='english')
doc_term_matrix = count_vect.fit_transform(title_selftext['data'].values.astype('U'))

from sklearn.decomposition import LatentDirichletAllocation

LDA = LatentDirichletAllocation(n_components=5, random_state=42)
LDA.fit(doc_term_matrix)

# np.set_printoptions(threshold=sys.maxsize)
# pprint(doc_term_matrix.toarray())

# for i in range(10):
#     random_id = random.randint(0,len(count_vect.get_feature_names()))
#     print(count_vect.get_feature_names()[random_id])


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [27]:
# Print words -- manually look at topics
first_topic = LDA.components_[0]
top_topic_words = first_topic.argsort()[-10:]

# for i in top_topic_words:
#     print(count_vect.get_feature_names()[i])

for i,topic in enumerate(LDA.components_):
    print(f'Top 10 words for topic #{i}:')
    print([count_vect.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print('\n')



Top 10 words for topic #0:
['speedrun', 'box', 'com', 've', 'run', 'just', 'speedrunning', 'http', 'runs', 'game']


Top 10 words for topic #1:
['guys', 'time', 'agdq', 'know', 'speed', 'speedrunning', 'just', 'games', 'game', 'run']


Top 10 words for topic #2:
['12', 'time', 'tv', 'new', 'race', 'www', 'twitch', 'mario', 'http', 'super']


Top 10 words for topic #3:
['10', 'help', 'speedrunning', 'www', 'like', 'http', 'game', 'cards', 'speedrun', 'marathon']


Top 10 words for topic #4:
['race', 'watch', 'time', 'https', 'twitch', 'run', 'com', 'youtube', 'www', 'wr']




In [28]:
# probability of all topics to each document
topic_values = LDA.transform(doc_term_matrix)
topic_values.shape

(412, 5)

In [30]:
# see the topic index with max value

#####This is for title + selftext#####
# submission_dataset_three['Topic'] = topic_values.argmax(axis=1)
# submission_dataset_three.head(10)

#####This is for title + selftext#####
title_selftext['Topic'] = topic_values.argmax(axis=1)
title_selftext.head(10)

,data,Topic
0,[WR]Hotline Miami Any% NG in 20:16 by Dingodrole,4
1,Couple questions about Chrono Trigger Any%1) W...,1
2,[WR] Luigi's Mansion 100% new WR by Veman300! ...,3
3,Jak and Daxter any% route?Does anyone know of ...,1
4,[WR] Castlevania:Harmony Of Dissonance Maxim A...,4
5,What do you consider some of the most impressi...,3
6,Rayman PSX (emulator) in 1:24:05,3
7,Anybody speedrun games for iOS/Android? Let's ...,1
8,SSX On Tour - The Peak Freeride in 14:00,0
9,Dark Souls 2 latest exe?Can someone post the l...,3
